In [2]:
#!/usr/bin/env python
# coding: utf-8
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [20]:
def read_article(file_name):
#     file = open(file_name, "r", encoding="utf-8")
    file = open(file_name, "r", encoding = "ISO-8859-1")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
#     sentences.pop() 
    
    return sentences

In [4]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [5]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [6]:
def generate_summary(file_name, top_n=5):
    nltk.download("stopwords")
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))

In [8]:
sent1 = ['India', 'is', 'a', 'powerful', 'democracy']
sent1 = [w.lower() for w in sent1]
sent2 = ['USA', 'has', 'a', 'powerful', 'economy']
sent2 = [w.lower() for w in sent2]
sent3 = ['China', 'is', 'the', 'most', 'populous', 'nation']
sent3 = [w.lower() for w in sent3]
print(sent1)
print(sent2)
print(sent3)

['india', 'is', 'a', 'powerful', 'democracy']
['usa', 'has', 'a', 'powerful', 'economy']
['china', 'is', 'the', 'most', 'populous', 'nation']


In [47]:
sentences = read_article('dummy_sample.txt')

India is a large democracy and a strong economy
USA has a strong economy and a powerful democracy
China is large with a strong economy; and the most populous nation, but not a democracy.


In [48]:
sentence_similarity(sent1, sent2, stopwords=None)

0.3999999999999999

In [49]:
sentence_similarity(sent2, sent3, stopwords=None)

0.0

In [29]:
sentence_similarity(sent3, sent1, stopwords=None)

0.18257418583505536

In [34]:
sentence_similarity(sentences[0], sentences[1], stopwords=None), sentence_similarity(sentences[0], sentences[2], stopwords=None)

(0.19999999999999996, 0.18257418583505536)

In [50]:
len(sentences), sentences[0], sentences[1], sentences[2]

(3,
 ['India', 'is', 'a', 'large', 'democracy', 'and', 'a', 'strong', 'economy'],
 ['USA', 'has', 'a', 'strong', 'economy', 'and', 'a', 'powerful', 'democracy'],
 ['China',
  'is',
  'large',
  'with',
  'a',
  'strong',
  'economy;',
  'and',
  'the',
  'most',
  'populous',
  'nation,',
  'but',
  'not',
  'a',
  'democracy.'])

In [55]:
stop_words = stopwords.words('english')
similarity_matrix = build_similarity_matrix(sentences, stop_words) 
similarity_matrix

array([[0.        , 0.6       , 0.3380617 ],
       [0.6       , 0.        , 0.16903085],
       [0.3380617 , 0.16903085, 0.        ]])

In [65]:
stop_words = stopwords.words('english')
sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words=[]) 
sentence_similarity_matrix

array([[0.        , 0.72727273, 0.56853524],
       [0.72727273, 0.        , 0.42640143],
       [0.56853524, 0.42640143, 0.        ]])

In [66]:
for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            sentence_similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
sentence_similarity_matrix

array([[0.        , 0.6       , 0.3380617 ],
       [0.6       , 0.        , 0.16903085],
       [0.3380617 , 0.16903085, 0.        ]])

In [70]:
sentence_similarity_matrix

array([[0.        , 0.6       , 0.3380617 ],
       [0.6       , 0.        , 0.16903085],
       [0.3380617 , 0.16903085, 0.        ]])

In [71]:
sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
sentence_similarity_graph

In [94]:
all_words = list(set(sent1 + sent2))
vector1 = [0] * len(all_words)
vector2 = [0] * len(all_words)

all_words, vector1, sent1
# build the vector for the first sentence
for w in sent1:
    if w in stop_words:
        continue
    vector1[all_words.index(w)] += 1

for w in sent2:
    if w in stop_words:
        continue
    vector2[all_words.index(w)] += 1
    
print(all_words)
print(vector1)
print(vector2)

['is', 'a', 'usa', 'economy', 'has', 'democracy', 'powerful', 'india']
[0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 1, 1, 0, 0, 1, 0]


In [124]:
vec1 = [1,1,1,1,0,0,0]
vec2 = [1,1,0,1,0,0,0]
dot_vec = np.dot(vec1, vec2)
# np.mod(vec1,vec2)
x = np.array([1,2,3,4,5])
mag1 = np.linalg.norm(x) # Magnitude of a Vector is independent of number of Zeros in the vector
mag1 = np.linalg.norm(vec1)
mag2 = np.linalg.norm(vec2)
print(mag1)
print(mag2)
print(dot_vec)
print(dot_vec / (mag1 * mag2)) # Similarity Score

2.0
1.7320508075688772
3
0.8660254037844387


In [122]:
# vector1.dot(vector2)
1 - cosine_distance(vec1, vec2)

0.8660254037844387

In [ ]:
# let's begin
generate_summary( "msft.txt", 2)

In [123]:
generate_summary( "dummy_sample.txt", 2)

[nltk_data] Downloading package stopwords to /home/nbuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
India is a large democracy and a strong economy
USA has a strong economy and a powerful democracy
China is large with a strong economy; and the most populous nation, but not a democracy.
Indexes of top ranked_sentence order are  [(0.41479975989973483, ['India', 'is', 'a', 'large', 'democracy', 'and', 'a', 'strong', 'economy']), (0.34388831612823245, ['USA', 'has', 'a', 'strong', 'economy', 'and', 'a', 'powerful', 'democracy']), (0.24131192397203255, ['China', 'is', 'large', 'with', 'a', 'strong', 'economy;', 'and', 'the', 'most', 'populous', 'nation,', 'but', 'not', 'a', 'democracy.'])]
Summarize Text: 
 India is a large democracy and a strong economy. USA has a strong economy and a powerful democracy
